# EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_train = pd.read_csv('data/train.csv.zip')

/tmp/ipykernel_23264/559265949.py:1: DtypeWarning: Columns (0,2,22,36,41,45,46,55,61,64,66) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_train = pd.read_csv('data/train.csv.zip')


In [3]:
raw_test = pd.read_csv('data/test.csv.zip')

/tmp/ipykernel_23264/1436528466.py:1: DtypeWarning: Columns (0,2,22,40,44,45,54,60,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_test = pd.read_csv('data/test.csv.zip')


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
raw_test.drop(columns=['listing_url','scrape_id','last_scraped',
              'name','description','neighborhood_overview','picture_url','host_id','host_url','host_name',
              'host_since','host_location','host_about','host_response_time','host_response_rate','host_acceptance_rate',
              'host_thumbnail_url','host_picture_url','host_neighbourhood','neighbourhood_cleansed',
              'neighbourhood_group_cleansed','bathrooms','minimum_nights','maximum_nights',
              'minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights',
              'minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','has_availability',
              'availability_30','availability_60','availability_90','availability_365','calendar_last_scraped',
              'number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','first_review',
              'last_review','review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
              'review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value',
              'license','instant_bookable','calculated_host_listings_count','calculated_host_listings_count_entire_homes',
              'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
              'reviews_per_month','last_searched','requires_license','region_id','region_name','region_parent_id',
              'region_parent_name','region_parent_parent_id','region_parent_parent_name'], axis=1, inplace=True)

In [12]:
raw_test.head()

,id,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities
0,0,f,1.0,1.0,"['email', 'phone', 'jumio', 'offline_governmen...",t,t,"Munich, Bayern, Germany",48.124790,11.597010,Entire rental unit,Entire home/apt,5.0,2 baths,1.0,4.0,"[""Shampoo"", ""Hair dryer"", ""Wifi"", ""Heating"", ""..."
1,1,f,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Paris, Île-de-France, France",48.892890,2.361360,Entire rental unit,Entire home/apt,2.0,1 bath,NaN,1.0,"[""Building staff"", ""Elevator"", ""Cooking basics..."
2,2,t,14.0,14.0,"['email', 'phone', 'work_email']",t,t,"Heraklion Crete, Greece",35.315311,25.238979,Entire residential home,Entire home/apt,8.0,1.5 baths,3.0,4.0,"[""Shampoo"", ""BBQ grill"", ""Private patio or bal..."
3,3,f,12.0,12.0,"['email', 'phone', 'offline_government_id', 's...",t,t,NaN,45.760790,4.834610,Private room in rental unit,Private room,3.0,1 bath,1.0,1.0,"[""Shampoo"", ""TV"", ""Wifi""]"
4,4,f,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"Barcelona, Catalunya, Spain",41.402520,2.140050,Private room in rental unit,Private room,2.0,1.5 shared baths,1.0,1.0,"[""Oven"", ""Coffee maker"", ""TV"", ""Patio or balco..."


In [14]:
raw_test.host_is_superhost.unique()

array(['f', 't', nan, '-9.11113', '-8.60792', '2.37446', '-8.31659',
       '-8.62264', '12.540807', '-8.28235', '-8.60773', '-9.10972'],
      dtype=object)

In [15]:
sample = pd.read_csv('data/sample.csv')

Cargados los datasets, vamos a echar un ojo a lo que tenemos en el train:

In [ ]:
sample.shape

In [ ]:
numericas = raw_train.select_dtypes(include=np.number) #mirar _get_numeric_data()

In [ ]:
numericas.shape

In [ ]:
#Hacemos el tts

In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
import h2o
from h2o.automl import H2OAutoML


In [ ]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

In [ ]:
data=numericas.sample(frac=0.95, random_state=786).reset_index(drop=True)

data_test=numericas.drop(data.index).reset_index(drop=True)

data.shape, data_test.shape

In [ ]:
train = h2o.H2OFrame(data)

In [ ]:
x = train.columns
y = "price"
x.remove(y)

In [ ]:
aml = H2OAutoML(max_models=4, seed=2)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)[:5]

In [ ]:
data_test=numericas.drop(data.index).reset_index(drop=True)
data_test=h2o.H2OFrame(data_test)
preds = aml.leader.predict(data_test)

In [ ]:
test_completo = pd.read_csv('data/test.csv')

In [ ]:
test_completo = h2o.H2OFrame(test_completo)

In [ ]:
prediccion_test_completo = aml.leader.predict(test_completo)

In [ ]:
sample = pd.read_csv('data/sample.csv')

In [ ]:
type(test_completo)

In [ ]:
test_pandas = h2o.as_list(test_completo)
type(test_pandas)

In [ ]:
type(respuestas)

In [ ]:
sample.shape

In [ ]:
sample.columns

In [ ]:
sample.sample()